## Exploratory Data Analysis 

In [1]:
#getting and working with data
import pickle

import pandas as pd
import numpy as np

import re
import string

In [2]:
# Reviews df
with open('/Users/jbpatty/project-4/reviews_compiled.pkl','rb') as read_file:
    reviews_compiled = pickle.load(read_file)

In [3]:
reviews_compiled.shape

(428177, 3)

In [4]:
# Note that the dataset has mostly positive reviews
reviews_compiled.sentiment.value_counts(normalize=True)

positive    0.914535
negative    0.085465
Name: sentiment, dtype: float64

In [5]:
reviews_compiled.sample(5)

,sentiment,review_text,asin
353261,positive,you almost cant find these bad boys anywhere a...,B0016MHL6W
256172,positive,I make tea with leave and I ground some down a...,B000UY86DE
566119,positive,My mom loves it. It's perfect for breakfast l...,B002SWB73C
442099,positive,great product fast delivery!,B001ACDOA0
328991,positive,Great deal as an add-on.,B0012OTF3Q


### Pre-Process the text

In [6]:
# Reviews - text preprocessing steps - remove numbers, captial letters, punctuation, and repetitive letters
def preprocess(docs):
    """
    Preprocess a corpus (Series) of documents before using a vectorizer
    - remove numbers and punctuation
    - convert all text to lower case
    """
    # Remove numbers and punctuation
    alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
    
    # Convert all text to lowercase
    punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
    
    # Remove repetitve letters 
    repeat_pattern = lambda x: re.sub(r'(\w)\1*', r'\1', x)
    
    return docs.map(alphanumeric).map(punc_lower).map(repeat_pattern)

In [ ]:
reviews_compiled['review_text'] = preprocess(reviews_compiled['review_text'])

In [ ]:
# Save to access later
reviews_compiled.to_pickle('/Users/jbpatty/project-4/reviews_compiled_EDA.to_pickle')

### Prepare data for modeling

In [ ]:
with open('/Users/jbpatty/project-4/reviews_compiled_EDA.to_pickle','rb') as read_file:
    reviews_compiled = pickle.load(read_file)

In [ ]:
# Split the data into X and y data sets
X = reviews_compiled.review_text
y = reviews_compiled.sentiment

In [ ]:
# Split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# The first document-term matrix has default Count Vectorizer values - counts of unigrams
from sklearn.feature_extraction.text import CountVectorizer

cv1 = CountVectorizer(stop_words='english')

X_train_cv1 = cv1.fit_transform(X_train)
X_test_cv1  = cv1.transform(X_test)

pd.DataFrame(X_train_cv1.toarray(), columns=cv1.get_feature_names()).head()

In [ ]:
# The second document-term matrix has both unigrams and bigrams, and indicators instead of counts
cv2 = CountVectorizer(ngram_range=(1,2), binary=True, stop_words='english')

X_train_cv2 = cv2.fit_transform(X_train)
X_test_cv2  = cv2.transform(X_test)

pd.DataFrame(X_train_cv2.toarray(), columns=cv2.get_feature_names()).head()

### Classifying using Logistic Regression

In [ ]:
# Create a logistic regression model to use
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [ ]:
# Train the first model
lr.fit(X_train_cv1, y_train)
y_pred_cv1 = lr.predict(X_test_cv1)

In [ ]:
# Train the second model
lr.fit(X_train_cv2, y_train)
y_pred_cv2 = lr.predict(X_test_cv2)

In [ ]:
# Create a function to calculate the error metrics, since we'll be doing this several times
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

def conf_matrix(actual, predicted):
    cm = confusion_matrix(actual, predicted)
    sns.heatmap(cm, xticklabels=['predicted_negative', 'predicted_positive'], 
                yticklabels=['actual_negative', 'actual_positive'], annot=True,
                fmt='d', annot_kws={'fontsize':20}, cmap="YlGnBu");

    true_neg, false_pos = cm[0]
    false_neg, true_pos = cm[1]

    accuracy = round((true_pos + true_neg) / (true_pos + true_neg + false_pos + false_neg),3)
    precision = round((true_pos) / (true_pos + false_pos),3)
    recall = round((true_pos) / (true_pos + false_neg),3)
    f1 = round(2 * (precision * recall) / (precision + recall),3)

    cm_results = [accuracy, precision, recall, f1]
    return cm_results

In [ ]:
# The heat map for the first logistic regression model
cm1 = conf_matrix(y_test, y_pred_cv1)

In [ ]:
# The heat map for the second logistic regression model
cm2 = conf_matrix(y_test, y_pred_cv2)

In [ ]:
# Compile all of the error metrics into a dataframe for comparison
results = pd.DataFrame(list(zip(cm1, cm2)))
results = results.set_index([['Accuracy', 'Precision', 'Recall', 'F1 Score']])
results.columns = ['LogReg1', 'LogReg2']
results

Comparing the two models, the second model outperforms the first model in all performance categories.

### Classifying using Naive Bayes

In [ ]:
# Fit the first Naive Bayes model
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train_cv1, y_train)

y_pred_cv1_nb = mnb.predict(X_test_cv1)

In [ ]:
# Fit the second Naive Bayes model
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
bnb.fit(X_train_cv2, y_train)

y_pred_cv2_nb = bnb.predict(X_test_cv2)

In [ ]:
# Here's the heat map for the first Naive Bayes model
cm3 = conf_matrix(y_test, y_pred_cv1_nb)

In [ ]:
# Here's the heat map for the second Naive Bayes model
cm4 = conf_matrix(y_test, y_pred_cv2_nb)

In [ ]:
# Compile all of the error metrics into a dataframe for comparison
results_nb = pd.DataFrame(list(zip(cm3, cm4)))
results_nb = results_nb.set_index([['Accuracy', 'Precision', 'Recall', 'F1 Score']])
results_nb.columns = ['NB1', 'NB2']
results_nb

results = pd.concat([results, results_nb], axis=1)
results

The second Logistic Regression model outperforms both Naive Bayes models.

In [ ]:
# Save to access later
results.to_pickle('/Users/jbpatty/project-4/results.to_pickle')

In [ ]:
with open('/Users/jbpatty/project-4/results.to_pickle','rb') as read_file:
    reviews_compiled = pickle.load(read_file)

### Using TF-IDF instead of Count Vectorizer

In [ ]:
# Create TF-IDF versions of the Count Vectorizers created earlier in the exercise
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf1 = TfidfVectorizer(stop_words='english')
X_train_tfidf1 = tfidf1.fit_transform(X_train)
X_test_tfidf1  = tfidf1.transform(X_test)

tfidf2 = TfidfVectorizer(ngram_range=(1,2), binary=True, stop_words='english')
X_train_tfidf2 = tfidf2.fit_transform(X_train)
X_test_tfidf2  = tfidf2.transform(X_test)

In [ ]:
# Fit the first logistic regression on the TF-IDF data
lr.fit(X_train_tfidf1, y_train)
y_pred_tfidf1_lr = lr.predict(X_test_tfidf1)
cm5 = conf_matrix(y_test, y_pred_tfidf1_lr)

In [ ]:
# Fit the second logistic regression on the TF-IDF data
lr.fit(X_train_tfidf2, y_train)
y_pred_tfidf2_lr = lr.predict(X_test_tfidf2)
cm6 = conf_matrix(y_test, y_pred_tfidf2_lr)

In [ ]:
# Fit the first Naive Bayes model on the TF-IDF data
mnb.fit(X_train_tfidf1.toarray(), y_train)
y_pred_tfidf1_nb = mnb.predict(X_test_tfidf1)
cm7 = conf_matrix(y_test, y_pred_tfidf1_nb)

In [ ]:
# Fit the second Naive Bayes model on the TF-IDF data
bnb.fit(X_train_tfidf2.toarray(), y_train)
y_pred_tfidf2_nb = bnb.predict(X_test_tfidf2)
cm8 = conf_matrix(y_test, y_pred_tfidf2_nb)

In [ ]:
# Compile all of the error metrics into a dataframe for comparison
results_tf = pd.DataFrame(list(zip(cm5, cm6, cm7, cm8)))
results_tf = results_tf.set_index([['Accuracy', 'Precision', 'Recall', 'F1 Score']])
results_tf.columns = ['LR1-TFIDF', 'LR2-TFIDF', 'NB1-TFIDF', 'NB2-TFIDF']
results_tf

results = pd.concat([results, results_tf], axis=1)
results

By using TF-IDF, I was able to improve the recall, but the accuracy and precision of the first Naive Bayes model, but not enought to outperforms the second Logistic Model.